In [ ]:
import http.server
import socket
import socketserver
import webbrowser
import pyqrcode
import os
import subprocess
import time
import requests
from urllib.parse import unquote

PORT = 8010
TEMP_FILE = "shared_text.txt"

# Ask user for input type
choice = input("Do you want to share a text message or a file? (text/file): ").strip().lower()

if choice == "text":
    text_content = input("Enter the text message to share: ")
    with open(TEMP_FILE, "w", encoding="utf-8") as f:
        f.write(text_content)
    file_to_serve = TEMP_FILE
    directory_to_serve = os.getcwd()  # Serve from current directory

elif choice == "file":
    file_path = input("Enter the file path to serve: ").strip('"')
    
    # Check if file exists
    if not os.path.isfile(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        exit(1)

    # Extract directory and filename
    directory_to_serve = os.path.dirname(file_path)
    file_to_serve = os.path.basename(file_path)

else:
    print("Invalid choice. Please enter 'text' or 'file'.")
    exit(1)

# Change working directory to the file's directory
os.chdir(directory_to_serve)

# Custom HTTP Handler that serves only the specified file
class CustomHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        requested_path = unquote(self.path.lstrip("/"))  # Decode URL-encoded path
        if requested_path != file_to_serve:
            self.send_error(404, "File not found")
        else:
            self.path = f"/{file_to_serve}"  # Serve only the specified file
            return super().do_GET()

# Start ngrok process
try:
    print("Starting ngrok... Please wait.")
    ngrok_process = subprocess.Popen(
        [r"C:\Users\SATHYA VARDHAN\Cyber project\ngrok_bin\ngrok.exe", "http", str(PORT)],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    time.sleep(5)  # Wait for ngrok to initialize

    # Fetch ngrok public URL from API
    ngrok_url = None
    try:
        response = requests.get("http://127.0.0.1:4040/api/tunnels")
        ngrok_data = response.json()
        ngrok_url = ngrok_data["tunnels"][0]["public_url"]
    except Exception as e:
        print(f"Error fetching ngrok URL: {e}")

    # Generate QR Code
    if ngrok_url:
        file_url = f"{ngrok_url}/{file_to_serve}"
        print(f"Public File URL: {file_url}")
        qr_code = pyqrcode.create(file_url, encoding='utf-8')

        qr_code.png("myqr.png", scale=8)
        webbrowser.open("myqr.png")
    else:
        print("Failed to generate ngrok URL. Falling back to local IP.")

        # Get local IP address
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        try:
            s.connect(("8.8.8.8", 80))
            local_ip = s.getsockname()[0]
        except:
            local_ip = "127.0.0.1"
        finally:
            s.close()

        local_url = f"http://{local_ip}:{PORT}/{file_to_serve}"
        qr_code = pyqrcode.create(local_url)
        qr_code.png("myqr.png", scale=8)
        webbrowser.open("myqr.png")

    # Serve the file
    with socketserver.TCPServer(("0.0.0.0", PORT), CustomHandler) as httpd:
        print(f"Serving '{file_to_serve}' at port {PORT}")
        if ngrok_url:
            print(f"Access file from any device: {file_url}")
        else:
            print(f"Access file on this network: {local_url}")
        print("Scan the QR code to access the file.")
        httpd.serve_forever()

except KeyboardInterrupt:
    print("\nShutting down server.")
finally:
    if 'ngrok_process' in locals():
        ngrok_process.terminate()

    # Clean up temporary file if text was shared
    if choice == "text" and os.path.exists(TEMP_FILE):
        os.remove(TEMP_FILE)


Do you want to share a text message or a file? (text/file):  text
Enter the text message to share:  dfghjk


Starting ngrok... Please wait.
Error fetching ngrok URL: HTTPConnectionPool(host='127.0.0.1', port=4040): Max retries exceeded with url: /api/tunnels (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E7EA52F8C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Failed to generate ngrok URL. Falling back to local IP.
Serving 'shared_text.txt' at port 8010
Access file on this network: http://192.168.29.14:8010/shared_text.txt
Scan the QR code to access the file.


In [1]:
# import http.server
# import socket
# import socketserver
# import webbrowser
# import pyqrcode
# import os
# import subprocess
# import time
# import requests
# from urllib.parse import unquote

# PORT = 8010
# TEMP_FILE = "shared_text.txt"

# # Ask user for input type
# choice = input("Do you want to share a text message or a file? (text/file): ").strip().lower()

# if choice == "text":
#     text_content = input("Enter the text message to share: ")
#     with open(TEMP_FILE, "w", encoding="utf-8") as f:
#         f.write(text_content)
#     file_to_serve = TEMP_FILE
#     directory_to_serve = os.getcwd()  # Serve from current directory

# elif choice == "file":
#     file_path = input("Enter the file path to serve: ").strip('"')
    
#     # Check if file exists
#     if not os.path.isfile(file_path):
#         print(f"Error: The file '{file_path}' does not exist.")
#         exit(1)

#     # Extract directory and filename
#     directory_to_serve = os.path.dirname(file_path)
#     file_to_serve = os.path.basename(file_path)

# else:
#     print("Invalid choice. Please enter 'text' or 'file'.")
#     exit(1)

# # Change working directory to the file's directory
# os.chdir(directory_to_serve)

# # Custom HTTP Handler that serves only the specified file
# class CustomHandler(http.server.SimpleHTTPRequestHandler):
#     def do_GET(self):
#         requested_path = unquote(self.path.lstrip("/"))  # Decode URL-encoded path
#         if requested_path != file_to_serve:
#             self.send_error(404, "File not found")
#         else:
#             self.path = f"/{file_to_serve}"  # Serve only the specified file
#             return super().do_GET()

# # Start ngrok process
# try:
#     print("Starting ngrok... Please wait.")
#     ngrok_process = subprocess.Popen(
#         ["C:\\ngrok\\ngrok.exe", "http", str(PORT)],
#         stdout=subprocess.PIPE,
#         stderr=subprocess.PIPE,
#         text=True
#     )
#     time.sleep(5)  # Wait for ngrok to initialize

#     # Fetch ngrok public URL from API
#     ngrok_url = None
#     try:
#         response = requests.get("http://127.0.0.1:4040/api/tunnels")
#         ngrok_data = response.json()
#         ngrok_url = ngrok_data["tunnels"][0]["public_url"]
#     except Exception as e:
#         print(f"Error fetching ngrok URL: {e}")

#     # Generate QR Code
#     if ngrok_url:
#         file_url = f"{ngrok_url}/{file_to_serve}"
#         print(f"Public File URL: {file_url}")
#         #qr_code = pyqrcode.create(file_url)
#         qr_code = pyqrcode.create(file_url, encoding='utf-8')

#         qr_code.png("myqr.png", scale=8)
#         webbrowser.open("myqr.png")
#     else:
#         print("Failed to generate ngrok URL. Falling back to local IP.")

#         # Get local IP address
#         s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
#         try:
#             s.connect(("8.8.8.8", 80))
#             local_ip = s.getsockname()[0]
#         except:
#             local_ip = "127.0.0.1"
#         finally:
#             s.close()

#         local_url = f"http://{local_ip}:{PORT}/{file_to_serve}"
#         qr_code = pyqrcode.create(local_url)
#         qr_code.png("myqr.png", scale=8)
#         webbrowser.open("myqr.png")

#     # Serve the file
#     with socketserver.TCPServer(("0.0.0.0", PORT), CustomHandler) as httpd:
#         print(f"Serving '{file_to_serve}' at port {PORT}")
#         if ngrok_url:
#             print(f"Access file from any device: {file_url}")
#         else:
#             print(f"Access file on this network: {local_url}")
#         print("Scan the QR code to access the file.")
#         httpd.serve_forever()

# except KeyboardInterrupt:
#     print("\nShutting down server.")
# finally:
#     if ngrok_process:
#         ngrok_process.terminate()

#     # Clean up temporary file if text was shared
#     if choice == "text" and os.path.exists(TEMP_FILE):
#         os.remove(TEMP_FILE)

Do you want to share a text message or a file? (text/file):  text
Enter the text message to share:  dfghj


Starting ngrok... Please wait.


NameError: name 'ngrok_process' is not defined